In [2]:
#Example of how to start using google bigquery
#json credential file used below is created at the google cloud platform console
#     Create Service Account, set role as owner, download json key file, link to that file below
#michaelcouch ~ % pip install google-cloud-bigquery
#michaelcouch ~% pip install pandas
#michaelcouch ~% pip install plotly==4.8.1
#michaelcouch ~% pip install plotly-geo     # Used for county level choropleth map
#virtuanenvname ~% pip install --user ipykernel
#virtualenvname ~% python -m ipykernel install --user --name=virtualevnname
#In jupyter notebook select vitualenvname as kernel

#Used for county level choropleth
#$ pip install geopandas==0.3.0
#$ pip install pyshp==1.2.10
#$ pip install shapely==1.6.3

import pandas as pd
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/michaelcouch/Documents/Data_Analytics/VirtualEnvZone/TestVenv1/dev-smoke-278920-cb67e104d184.json"
# The below lines allow for file paths to be shared on localhost and pythonanywhere server __file__ is a python majic variable 
#THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))
#my_file = os.path.join(THIS_FOLDER,"../dev-smoke-278920-cb67e104d184.json")
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = my_file

from google.cloud import bigquery
client = bigquery.Client()

In [3]:
import pandas as pd
from random import random
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.figure_factory as ff # For County level Choropleth Map

In [4]:
query="""select * from bigquery-public-data.covid19_usafacts.confirmed_cases 
    WHERE state='WA'
    """
query_job = client.query(query)
casesDf=query_job.to_dataframe()
casesDf.head()

,county_fips_code,county_name,state,state_fips_code,_1_22_20,_1_23_20,_1_24_20,_1_25_20,_1_26_20,_1_27_20,...,_10_26_20,_10_27_20,_10_28_20,_10_29_20,_10_30_20,_10_31_20,_11_1_20,_11_2_20,_11_3_20,_11_4_20
0,53029,Island County,WA,53,0,0,0,0,0,0,...,377,382,382,384,393,399,404,408,412,414
1,53033,King County,WA,53,1,1,1,1,1,1,...,26519,26599,26789,27021,27353,27735,27951,28223,28337,28926
2,53053,Pierce County,WA,53,0,0,0,0,0,0,...,9971,10027,10116,10228,10331,10431,10514,10616,10685,10879
3,53057,Skagit County,WA,53,0,0,0,0,0,0,...,1241,1246,1251,1256,1261,1265,1287,1296,1304,1324
4,53061,Snohomish County,WA,53,0,0,0,0,0,0,...,8526,8629,8705,8787,8881,8960,9024,9125,9233,9359


In [5]:
# Transpose state data for graphing
casesDf_transp=casesDf.T
casesDf_transp.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
county_fips_code,53029,53033,53053,53057,53061,00000,53005,53017,53021,53001,...,53007,53037,53031,53011,53063,53077,53067,53073,53025,53035
county_name,Island County,King County,Pierce County,Skagit County,Snohomish County,Statewide Unallocated,Benton County,Douglas County,Franklin County,Adams County,...,Chelan County,Kittitas County,Jefferson County,Clark County,Spokane County,Yakima County,Thurston County,Whatcom County,Grant County,Kitsap County
state,WA,WA,WA,WA,WA,WA,WA,WA,WA,WA,...,WA,WA,WA,WA,WA,WA,WA,WA,WA,WA
state_fips_code,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53
_1_22_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
	casesDf_transp.columns = casesDf_transp.loc['county_name']  # set county names as column names
	wa_covid_graph_data = casesDf_transp.drop(['county_name', 'state', 'county_fips_code', 'state_fips_code']) # drop rows by name
wa_covid_graph_data.head()

county_name,Island County,King County,Pierce County,Skagit County,Snohomish County,Statewide Unallocated,Benton County,Douglas County,Franklin County,Adams County,...,Chelan County,Kittitas County,Jefferson County,Clark County,Spokane County,Yakima County,Thurston County,Whatcom County,Grant County,Kitsap County
_1_22_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_1_23_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_1_24_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_1_25_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_1_26_20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#turn the index into a string to convert to datetime

#chops off first character of index, then converts all underscores _ to slashes /, then converts to date time and
#sets back on index
wa_covid_graph_data.index = pd.to_datetime(wa_covid_graph_data.index.str[1:].str.replace('_','/'))

In [8]:
# Build the graph and show it
xAxis=wa_covid_graph_data.index # set x axis to dates (row indexes)
yAxis=wa_covid_graph_data['King County']   # set y axis to values in King County
fig=go.Figure(data=go.Scatter(x=xAxis, y=yAxis))

#Add titles and legends
fig.update_layout(annotations=[dict(text='Source: USAFacts COVID19 dataset, retrieved from bigquery-public-data.covid19_usafacts.confirmed_cases', 
                                    showarrow=False, 
                                    xref='paper', 
                                    yref='paper', 
                                    x=1, 
                                    y=-.1)],
                 title="Covid Cases")
plt_div = plot(fig)

In [9]:
#Build a map base layer
fig = go.Figure(go.Scattergeo())
fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Blue"
)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
plt_div = plot(fig)

In [10]:
#TODO - get state totals on last recorded day, keep FIPs codes and create a chloropleth map of states, 
#then do a county level using legacy plotly's legacy figure factory

#lastRow = casesDf_transp.iloc[[0,-1]] # assign first row and last row to lastRow df
#lastRow.head()

In [11]:
# Get all the data (all states)
query="""select * from bigquery-public-data.covid19_usafacts.confirmed_cases
    """
query_job = client.query(query)
casesDf=query_job.to_dataframe()
casesDf.head()


,county_fips_code,county_name,state,state_fips_code,_1_22_20,_1_23_20,_1_24_20,_1_25_20,_1_26_20,_1_27_20,...,_10_26_20,_10_27_20,_10_28_20,_10_29_20,_10_30_20,_10_31_20,_11_1_20,_11_2_20,_11_3_20,_11_4_20
0,01073,Jefferson County,AL,01,0,0,0,0,0,0,...,23443,23573,23769,23940,24088,24285,24472,24597,24754,24973
1,04013,Maricopa County,AZ,04,0,0,0,0,1,1,...,154722,155458,155951,156736,157728,158761,159781,160184,161154,161588
2,00000,Statewide Unallocated,AR,05,0,0,0,0,0,0,...,2205,2098,2095,2085,2115,2111,1640,1675,1738,1604
3,06001,Alameda County,CA,06,0,0,0,0,0,0,...,23391,23391,23576,23636,23775,23876,23994,24095,24162,24233
4,06013,Contra Costa County,CA,06,0,0,0,0,0,0,...,18763,18763,18877,18981,19086,19195,19254,19337,19408,19517


In [12]:
# Transform the full data set to state level and prepare for graph

# Get state totals by state_fips_code AND transpose
state_levelDf=casesDf.groupby(['state']).sum().T
#state_levelDf.head()

# keep only columns and last row (running total)
state_levelDf_total = state_levelDf.iloc[[-1]]
state_levelDf_total.head()
statesDf=state_levelDf_total.T
statesDf.columns = ['Total']
statesDf.head()

# Get the date of the last record and chop off the first character of the string, used in layout
lastUpdated = list(casesDf.columns)[-1][1:].replace('_','/')
statesDf['Total']

state
AK     16764
AL    197777
AR    115812
AZ    250633
CA    951000
CO    117637
CT     75373
DC     17601
DE     25533
FL    821123
GA    366452
HI     15473
IA    136047
ID     68314
IL    437544
IN    194556
KS     93653
KY    113009
LA    185144
MA    169726
MD    148766
ME      7077
MI    212160
MN    160923
MO    196576
MS    123887
MT     35955
NC    283968
ND     49838
NE     75267
NH     11563
NJ    245257
NM     50251
NV    103984
NY    515884
OH    230210
OK    127772
OR     47049
PA    217666
RI     32657
SC    180870
SD     49791
TN    269802
TX    939064
UT    121614
VA    185836
VT      2267
WA    111480
WI    257288
WV     25987
WY     15044
Name: Total, dtype: int64

In [13]:
# Create chloropleth graph map of states with last date collected as the color
fig = go.Figure(data=go.Choropleth(
    locations=statesDf.index, # Spatial coordinates
    z = statesDf['Total'], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Positive Cases",
))

fig.update_layout(
    title_text = 'Total Positive Covid Cases Per State as of {}'.format(lastUpdated),
    geo_scope='north america', # limit map scope to USA
)

StatesMapPlotDiv=plot(fig)

In [14]:
#Create a line graph of all the states over time
# Prepare the data here, graph in next cell

# Use state level df to create a total column, will be used for line graph of US states
state_levelDf['Total'] = state_levelDf.sum(axis=1)

#Convert index into date
state_levelDf.index = pd.to_datetime(state_levelDf.index.str.replace('_','/'))

# Investigate results at this csv if desired
#state_levelDf.to_csv('state_levelDf.csv')

In [15]:
# Build the line graph and show it
xAxis=state_levelDf.index # set x axis to dates (row indexes)
yAxis=state_levelDf['Total']   # set y axis to values in King County
fig=go.Figure(data=go.Scatter(x=xAxis, y=yAxis))

#Add titles and legends
fig.update_layout(annotations=[dict(text='Source: USAFacts COVID19 dataset, retrieved from bigquery-public-data.covid19_usafacts.confirmed_cases', 
                                    showarrow=False, 
                                    xref='paper', 
                                    yref='paper', 
                                    x=1, 
                                    y=-.1)],
                 title="Total US Covid Cases")
plt_div = plot(fig)

In [16]:
# Build county level choropleth map
# First, get county level data for last recorded day

#keep only last date recorded and county info (first couple rows)
lastCountyData = casesDf.T.iloc[[0,1,2,-1]].T

#Rename last column to "Total"
lastCountyData.rename(columns = {list(lastCountyData)[-1]:'Total'}, inplace=True)
lastCountyData.head()

,county_fips_code,county_name,state,Total
0,01073,Jefferson County,AL,24973
1,04013,Maricopa County,AZ,161588
2,00000,Statewide Unallocated,AR,1604
3,06001,Alameda County,CA,24233
4,06013,Contra Costa County,CA,19517


In [17]:
CountyChor = ff.create_choropleth(
    fips=lastCountyData['county_fips_code'], values=lastCountyData['Total'], scope=['usa'],
    show_state_data=False,
    show_hover=True,
    asp = 2.9,
    title_text = 'USA by Unemployment %',
    legend_title = '% unemployed'
)
CountyChor.layout.template = None
county_chor_div = plot(CountyChor)

/Users/michaelcouch/Library/Python/2.7/lib/python/site-packages/plotly/figure_factory/_county_choropleth.py:838: UserWarning:

Unrecognized FIPS Values

Whoops! It looks like you are trying to pass at least one FIPS value that is not in our shapefile of FIPS and data for the counties. Your choropleth will still show up but these counties cannot be shown.
Unrecognized FIPS are: [0, 0, 0, 0, 0, 6000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



## Create a line graph to show daily NEW case count

In [18]:
# Start at the beginning query of all cases and build a dataframe that has total per date and new cases since last row
casesDf.head()

,county_fips_code,county_name,state,state_fips_code,_1_22_20,_1_23_20,_1_24_20,_1_25_20,_1_26_20,_1_27_20,...,_10_26_20,_10_27_20,_10_28_20,_10_29_20,_10_30_20,_10_31_20,_11_1_20,_11_2_20,_11_3_20,_11_4_20
0,01073,Jefferson County,AL,01,0,0,0,0,0,0,...,23443,23573,23769,23940,24088,24285,24472,24597,24754,24973
1,04013,Maricopa County,AZ,04,0,0,0,0,1,1,...,154722,155458,155951,156736,157728,158761,159781,160184,161154,161588
2,00000,Statewide Unallocated,AR,05,0,0,0,0,0,0,...,2205,2098,2095,2085,2115,2111,1640,1675,1738,1604
3,06001,Alameda County,CA,06,0,0,0,0,0,0,...,23391,23391,23576,23636,23775,23876,23994,24095,24162,24233
4,06013,Contra Costa County,CA,06,0,0,0,0,0,0,...,18763,18763,18877,18981,19086,19195,19254,19337,19408,19517


In [19]:
# Sum all cases for each day then transpose so days are rows
cases_total_per_day = casesDf.groupby([True]*len(casesDf)).agg('sum').T

In [20]:
cases_total_per_day.tail(15)

,True
_10_21_20,8265015
_10_22_20,8342764
_10_23_20,8424968
_10_24_20,8496299
_10_25_20,8559222
_10_26_20,8627232
_10_27_20,8698461
_10_28_20,8776992
_10_29_20,8866402
_10_30_20,8962804


In [21]:
# Calculate the volume difference from previous row (new cases each day) for entire country

new_cases_per_day = cases_total_per_day.diff()
new_cases_per_day['Date'] = pd.to_datetime(new_cases_per_day.index.str.replace('_','/'))
new_cases_per_day.tail()

,True,Date
_10_31_20,73686.0,2020-10-31
_11_1_20,82781.0,2020-11-01
_11_2_20,85948.0,2020-11-02
_11_3_20,91994.0,2020-11-03
_11_4_20,107711.0,2020-11-04


### The line graph of new cases per day for entire country

In [22]:
# Build the line graph and show it
xAxis=new_cases_per_day['Date'] # set x axis to dates (row indexes)
yAxis=new_cases_per_day[True]   # set y axis to values in King County
fig=go.Figure(data=go.Scatter(x=xAxis, y=yAxis))

#Add titles and legends
fig.update_layout(annotations=[dict(text='Source: USAFacts COVID19 dataset, retrieved from bigquery-public-data.covid19_usafacts.confirmed_cases', 
                                    showarrow=False, 
                                    xref='paper', 
                                    yref='paper', 
                                    x=1, 
                                    y=-.1)],
                 title="Daily New Covid Cases")
plt_div = plot(fig)

## Create a state map of most recent daily new case count
### Get the most recent daily new case count

In [23]:
# Sum all cases for each day then transpose so days are rows
state_per_day = casesDf.groupby(['state']).agg('sum').T

state_per_day = state_per_day.diff()
state_per_day['Date'] = pd.to_datetime(state_per_day.index.str.replace('_','/'))

# Select select the last row and chop off the date, this is the most recent daily new case count for each state
state_recent_new_count = state_per_day.iloc[-1][:-1]


In [24]:
# Create chloropleth graph map of states with last date collected as the color
fig = go.Figure(data=go.Choropleth(
    locations=state_recent_new_count.index, # Spatial coordinates
    z = state_recent_new_count, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "New Cases",
))

fig.update_layout(
    title_text = 'New Cases Per Day as of {}'.format(lastUpdated),
    geo_scope='usa', # limit map scope to USA
)

StatesMapPlotDiv=plot(fig)

## Per Population Map

In [30]:
popdf = pd.read_csv('./2019_State_Census_Population.csv')
popdf.head()

,State,Initial,Population
0,Alabama,AL,4903185
1,Alaska,AK,731545
2,Arizona,AZ,7278717
3,Arkansas,AR,3017804
4,California,CA,39512223


In [31]:
query="""select * FROM dev-smoke-278920.COVID_TEST_DATASET.2019_State_Census_Population
    """
query_job = client.query(query)
popdf=query_job.to_dataframe()
popdf.head()

,State,Initial,Population
0,Alabama,AL,4903185
1,Alaska,AK,731545
2,Arizona,AZ,7278717
3,Arkansas,AR,3017804
4,California,CA,39512223


In [32]:
statesDf['Initial']=statesDf.index
statesDf.head()

,Total,Initial
state,,
AK,16764,AK
AL,197777,AL
AR,115812,AR
AZ,250633,AZ
CA,951000,CA


In [33]:
states_with_pop = pd.merge(statesDf, popdf)
states_with_pop.head()

,Total,Initial,State,Population
0,16764,AK,Alaska,731545
1,197777,AL,Alabama,4903185
2,115812,AR,Arkansas,3017804
3,250633,AZ,Arizona,7278717
4,951000,CA,California,39512223


In [34]:
states_with_pop['case_to_pop_percent'] = states_with_pop['Total']/states_with_pop['Population']*100
states_with_pop.head()

,Total,Initial,State,Population,case_to_pop_percent
0,16764,AK,Alaska,731545,2.291588
1,197777,AL,Alabama,4903185,4.033643
2,115812,AR,Arkansas,3017804,3.837625
3,250633,AZ,Arizona,7278717,3.443368
4,951000,CA,California,39512223,2.406850


In [35]:
# Create chloropleth graph map of states with last date collected as the color
fig = go.Figure(data=go.Choropleth(
    locations=states_with_pop['Initial'], # Spatial coordinates
    z = states_with_pop['case_to_pop_percent'], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "% of Population",
))

fig.update_layout(
    title_text = 'Cases as a % of State Population (US Census 2019)',
    geo_scope='usa', # limit map scope to USA
)

StatesMapPlotDiv=plot(fig)